In [7]:
%reload_ext autoreload
%autoreload 2
import sys
import numpy as np
import arrow
import torch

sys.path.append('B://workspace/tensorflow')

from src.config.params import Areas, Area
from src.utils.util import scope
from src.config.params import MODEL_SAVE_PATH

resolution = 2
seq_len = 2
offset = 240
start_time = arrow.get('2004-01-01').shift(months=offset)

print(f"start_time: {start_time.format('YYYY-MM-DD')}")


area = Area(title=f"global_conv_lstm", lon=[-180, 180], lat=[-80, 80], description="Global")

# area = Areas[0]

# area_blocks = scope()
area_blocks = [area]

start_time: 2024-01-01


In [9]:
def train_area(area, pre_model=None):
    
    from src.trainer.lstm import LSTMTrainer

    model_params = {
        'input_dim': 1,  # 输入通道数
        'hidden_dim': 10,  # 隐藏层通道数
        'kernel_size': (3, 3),  # 卷积核大小
        'num_layers': 1,  # LSTM层数
        'bias': True  # 是否使用偏置
    }

    trainer = LSTMTrainer(
        "LSTM 训练器",
        area,
        seq_len=seq_len,
        offset=0,
        resolution=resolution,
        epochs=500,
        batch_size=35,
        pre_model=pre_model,
        **model_params,
    )

    model = trainer.train()
    
    if pre_model is None:
        torch.save(model, f"{MODEL_SAVE_PATH}/{area.title}.pkl")
    else:
        torch.save(model, f"{MODEL_SAVE_PATH}/{area.title}_new.pkl")
    
    return model
    


In [10]:
def plot_model_prediction(sst_model, area, off):
    from numpy import sqrt, nanmean
    from torch.utils.data import DataLoader
    
    from src.dataset.ERA5 import ERA5SSTMonthlyDataset
    from src.plot.sst import plot_sst, plot_sst_diff

    # sst_model = load(f"{model_path}")

    lon = area.lon
    lat = area.lat
    
    o = off or offset

    dataset = ERA5SSTMonthlyDataset(width=seq_len, offset=o, resolution=resolution, lon=lon, lat=lat)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    it = iter(loader)

    input_, output = next(it)
    
    print(input_.shape)

    pred = sst_model(input_)

    pred = pred.detach().numpy()[0, 0, :, :]
    output = output.detach().numpy()[0, 0, :, :]

    plot_sst(output, area.lon, area.lat, resolution, area.title, title=f'{area.title} origin')
    plot_sst(pred, area.lon, area.lat, resolution, area.title, title=f'{area.title} prediction')

    pred_diff = pred - output
    
    plot_sst_diff(pred_diff, area.lon, area.lat, resolution, area.title, title=f'{area.title} error')

    # 计算 RMSE
    rmse = sqrt(nanmean((pred - output) ** 2))

    print(f"预测海表月平均误差: {rmse}")

    # profile, origin_profile, rmses_profile  = predict_rf(rf_model, area, pred, 236)

    # print(f"预测剖面误差: {rmses_profile}")

    # plot_sst_comparison(mean_monthly_sst, pred, area.lon, area.lat, title="2023-10 ERA5")


In [11]:
import os
# 开始训练
start = 0

area_blocks = area_blocks[start:]

for area in area_blocks:
    pre_model = None
    
    if (os.path.exists(f'{MODEL_SAVE_PATH}/{area.title}.pkl')):
        pre_model = torch.load(f'{MODEL_SAVE_PATH}/{area.title}.pkl')
        print(f'{area.title} 预训练模型加载成功')
    
    sst_model = train_area(area, pre_model)

起始时间：2004-01-01


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type        | Params | Mode 
--------------------------------------------------
0 | cell_list | ModuleList  | 9.9 K  | train
1 | fc        | Sequential  | 2.6 M  | train
2 | nor       | BatchNorm2d | 20     | train
--------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.281    Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

b:\workspace\tensorflow\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
B:\/workspace/tensorflow\src\dataset\ERA5.py:201: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  sst_time_series = tensor(sst_time_series, requires_grad=True)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]DEBUG:      0
DEBUG:      torch.Size([35, 1, 1, 80, 180])


RuntimeError: running_mean should contain 16 elements not 10